In [ ]:
import requests
import json
import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon
import pandas as pd
import time
from typing import Dict, List, Any

def build_overpass_query() -> str:
    """
    Construye la consulta Overpass para obtener áreas protegidas del Perú.
    Sigue las mejores prácticas mencionadas en la documentación de OSM API.
    
    Returns:
        str: Consulta Overpass formateada
    """
    # Bbox aproximado de Perú: [sur, oeste, norte, este]
    peru_bbox = "-18.5,-81.5,0.5,-68.5"
    
    query = f"""
    [out:json][timeout:300][maxsize:1073741824][bbox:{peru_bbox}];
    (
      // Consulta específica para áreas protegidas siguiendo estándares OSM
      // Relaciones con boundary=protected_area (estándar principal)
      relation["boundary"="protected_area"];
      
      // Ways que forman límites de áreas protegidas
      way["boundary"="protected_area"];
      
      // Reservas naturales (clasificación leisure)
      relation["leisure"="nature_reserve"];
      way["leisure"="nature_reserve"];
      
      // Cualquier elemento con clasificación de protección IUCN
      relation["protect_class"~"^[1-6]$|^I[I-V]?$"];
      way["protect_class"~"^[1-6]$|^I[I-V]?$"];
      
      // Elementos específicos para Perú con designation
      relation["designation"~"Parque Nacional|Reserva Nacional|Santuario Nacional|Santuario Histórico|Reserva Paisajística|Refugio de Vida Silvestre|Bosque de Protección|Coto de Caza|Zona Reservada|Área de Conservación"];
      way["designation"~"Parque Nacional|Reserva Nacional|Santuario Nacional|Santuario Histórico|Reserva Paisajística|Refugio de Vida Silvestre|Bosque de Protección|Coto de Caza|Zona Reservada|Área de Conservación"];
    );
    (._;>;);
    out geom;
    """
    return query

def fetch_osm_data(query: str, max_retries: int = 3) -> Dict[str, Any]:
    """
    Obtiene datos de OSM usando la API Overpass siguiendo buenas prácticas.
    
    Args:
        query: Consulta Overpass
        max_retries: Número máximo de reintentos
        
    Returns:
        Dict con los datos de respuesta de OSM
    """
    # Lista de servidores Overpass para balancear carga
    overpass_urls = [
        "https://overpass-api.de/api/interpreter",
        "https://overpass.kumi.systems/api/interpreter",
        "https://overpass.openstreetmap.ru/api/interpreter"
    ]
    
    # Headers para identificar el cliente
    headers = {
        'User-Agent': 'Peru Protected Areas Study/1.0 (Educational Purpose)'
    }
    
    for attempt in range(max_retries):
        # Rotar entre servidores
        url = overpass_urls[attempt % len(overpass_urls)]
        
        try:
            print(f"Realizando consulta a Overpass API (intento {attempt + 1}/{max_retries})...")
            print(f"Servidor: {url}")
            
            # Respetar límites de la API con timeout apropiado
            response = requests.post(
                url, 
                data=query, 
                headers=headers,
                timeout=300
            )
            response.raise_for_status()
            
            data = response.json()
            print(f"Consulta exitosa. Elementos obtenidos: {len(data.get('elements', []))}")
            return data
            
        except requests.exceptions.RequestException as e:
            print(f"Error en intento {attempt + 1} con {url}: {e}")
            if attempt < max_retries - 1:
                wait_time = 5 * (attempt + 1)  # Backoff exponencial
                print(f"Esperando {wait_time} segundos antes del siguiente intento...")
                time.sleep(wait_time)
            else:
                raise
    
    return {}

def filter_protected_areas(elements: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    Filtra las áreas protegidas según criterios específicos.
    
    Criterios de filtrado:
    - protect_class en {"1", "2", "3", "4", "5", "6"} (IUCN categories)
    - boundary=protected_area
    - leisure=nature_reserve
    
    Args:
        elements: Lista de elementos de OSM
        
    Returns:
        Lista filtrada de elementos
    """
    # Clases de protección aceptadas (IUCN y otras categorías relevantes)
    accepted_protect_classes = {"1", "2", "3", "4", "5", "6", "1a", "1b", "II", "III", "IV", "V", "VI"}
    
    filtered_elements = []
    
    for element in elements:
        tags = element.get('tags', {})
        
        # Criterios de filtrado
        has_boundary_protected = tags.get('boundary') == 'protected_area'
        has_leisure_nature = tags.get('leisure') == 'nature_reserve'
        has_valid_protect_class = tags.get('protect_class', '').lower() in [pc.lower() for pc in accepted_protect_classes]
        
        # Filtros adicionales para parques nacionales, reservas de vida silvestre y forestales
        designation = tags.get('designation', '').lower()
        name = tags.get('name', '').lower()
        
        is_national_park = 'parque nacional' in designation or 'national park' in designation
        is_wildlife_reserve = 'reserva' in designation and ('vida silvestre' in designation or 'wildlife' in designation)
        is_forest_reserve = 'reserva forestal' in designation or 'forest reserve' in designation
        is_nature_reserve = 'reserva natural' in designation or 'nature reserve' in designation
        
        # Aplicar filtros
        if (has_boundary_protected or has_leisure_nature or has_valid_protect_class or 
            is_national_park or is_wildlife_reserve or is_forest_reserve or is_nature_reserve):
            
            filtered_elements.append(element)
            print(f"Aceptado: {tags.get('name', 'Sin nombre')} - "
                  f"protect_class: {tags.get('protect_class', 'N/A')}, "
                  f"designation: {tags.get('designation', 'N/A')}")
    
    print(f"\nElementos filtrados: {len(filtered_elements)} de {len(elements)} originales")
    return filtered_elements

def create_polygon_from_way(way: Dict[str, Any]) -> Polygon:
    """
    Crea un polígono a partir de un way de OSM.
    
    Args:
        way: Elemento way de OSM con geometría
        
    Returns:
        Shapely Polygon
    """
    if 'geometry' not in way:
        return None
        
    coords = [(node['lon'], node['lat']) for node in way['geometry']]
    
    # Asegurar que el polígono esté cerrado
    if len(coords) > 2 and coords[0] != coords[-1]:
        coords.append(coords[0])
    
    if len(coords) >= 4:  # Mínimo para un polígono válido
        return Polygon(coords)
    
    return None

def create_multipolygon_from_relation(relation: Dict[str, Any], ways_dict: Dict[int, Dict]) -> MultiPolygon:
    """
    Crea un MultiPolygon a partir de una relación de OSM.
    
    Args:
        relation: Elemento relation de OSM
        ways_dict: Diccionario de ways indexado por ID
        
    Returns:
        Shapely MultiPolygon o Polygon
    """
    outer_ways = []
    inner_ways = []
    
    for member in relation.get('members', []):
        if member['type'] == 'way' and member['ref'] in ways_dict:
            way = ways_dict[member['ref']]
            if member.get('role') == 'outer':
                outer_ways.append(way)
            elif member.get('role') == 'inner':
                inner_ways.append(way)
    
    polygons = []
    for way in outer_ways:
        poly = create_polygon_from_way(way)
        if poly:
            polygons.append(poly)
    
    if polygons:
        if len(polygons) == 1:
            return polygons[0]
        else:
            return MultiPolygon(polygons)
    
    return None

def convert_to_geodataframe(elements: List[Dict[str, Any]]) -> gpd.GeoDataFrame:
    """
    Convierte elementos de OSM a GeoDataFrame.
    
    Args:
        elements: Lista de elementos filtrados de OSM
        
    Returns:
        GeoDataFrame con las áreas protegidas
    """
    rows = []
    ways_dict = {}
    
    # Crear diccionario de ways para las relaciones
    for element in elements:
        if element['type'] == 'way':
            ways_dict[element['id']] = element
    
    for element in elements:
        tags = element.get('tags', {})
        geometry = None
        
        if element['type'] == 'way':
            geometry = create_polygon_from_way(element)
        elif element['type'] == 'relation':
            geometry = create_multipolygon_from_relation(element, ways_dict)
        
        if geometry and not geometry.is_empty:
            row = {
                'osm_id': element['id'],
                'osm_type': element['type'],
                'name': tags.get('name', ''),
                'name_es': tags.get('name:es', ''),
                'designation': tags.get('designation', ''),
                'protect_class': tags.get('protect_class', ''),
                'boundary': tags.get('boundary', ''),
                'leisure': tags.get('leisure', ''),
                'operator': tags.get('operator', ''),
                'website': tags.get('website', ''),
                'wikidata': tags.get('wikidata', ''),
                'area_km2': geometry.area * 111.32 * 111.32,  # Aproximación área en km²
                'geometry': geometry
            }
            rows.append(row)
    
    if rows:
        gdf = gpd.GeoDataFrame(rows, crs='EPSG:4326')
        return gdf
    else:
        # Crear GeoDataFrame vacío con las columnas correctas
        return gpd.GeoDataFrame(columns=[
            'osm_id', 'osm_type', 'name', 'name_es', 'designation', 
            'protect_class', 'boundary', 'leisure', 'operator', 
            'website', 'wikidata', 'area_km2', 'geometry'
        ], crs='EPSG:4326')

def main():
    """
    Función principal que ejecuta todo el proceso.
    """
    print("=== Adquisición de Áreas Protegidas del Perú desde OSM ===\n")
    
    # 1. Construir consulta Overpass
    print("1. Construyendo consulta Overpass...")
    query = build_overpass_query()
    print("Consulta Overpass:")
    print(query)
    print()
    
    # 2. Obtener datos de OSM
    print("2. Obteniendo datos de OSM...")
    try:
        data = fetch_osm_data(query)
        elements = data.get('elements', [])
    except Exception as e:
        print(f"Error al obtener datos de OSM: {e}")
        
        # Intentar con una consulta más simple si falla
        print("\nIntentando con consulta simplificada...")
        simple_query = """
        [out:json][timeout:300][bbox:-18.5,-81.5,0.5,-68.5];
        (
          relation["boundary"="protected_area"];
          way["boundary"="protected_area"];
        );
        out geom;
        """
        try:
            data = fetch_osm_data(simple_query)
            elements = data.get('elements', [])
        except Exception as e2:
            print(f"Error también con consulta simplificada: {e2}")
            return
    
    if not elements:
        print("No se obtuvieron datos de OSM.")
        print("Esto puede deberse a:")
        print("- Falta de datos de áreas protegidas en OSM para Perú")
        print("- Problemas de conectividad con la API Overpass")
        print("- Filtros demasiado restrictivos")
        return
    
    # 3. Filtrar áreas protegidas
    print("3. Filtrando áreas protegidas...")
    filtered_elements = filter_protected_areas(elements)
    
    if not filtered_elements:
        print("No se encontraron áreas protegidas después del filtrado.")
        print("Elementos originales encontrados:")
        for element in elements[:5]:  # Mostrar primeros 5
            tags = element.get('tags', {})
            print(f"- ID: {element['id']}, Tipo: {element['type']}")
            print(f"  Tags: {dict(list(tags.items())[:3])}")  # Primeros 3 tags
        return
    
    # 4. Convertir a GeoDataFrame
    print("4. Convirtiendo a GeoDataFrame...")
    gdf = convert_to_geodataframe(filtered_elements)
    
    if gdf.empty:
        print("No se pudieron crear geometrías válidas.")
        return
    
    # 5. Mostrar estadísticas
    print(f"\n=== Estadísticas ===")
    print(f"Total de áreas protegidas: {len(gdf)}")
    print(f"Área total aproximada: {gdf['area_km2'].sum():.2f} km²")
    print(f"Área promedio: {gdf['area_km2'].mean():.2f} km²")
    
    print(f"\nDistribución por tipo de protección:")
    print(gdf['protect_class'].value_counts())
    
    print(f"\nPrimeras 5 áreas por tamaño:")
    top_areas = gdf.nlargest(5, 'area_km2')[['name', 'designation', 'area_km2']]
    print(top_areas.to_string(index=False))
    
    # 6. Guardar en GeoPackage
    print("\n5. Guardando en GeoPackage...")
    output_file = "areas_protegidas_solo_peru.gpkg"
    
    try:
        gdf.to_file(output_file, driver="GPKG")
        print(f"Archivo guardado exitosamente: {output_file}")
        
        # Verificar el archivo guardado
        gdf_loaded = gpd.read_file(output_file)
        print(f"Verificación: {len(gdf_loaded)} áreas protegidas guardadas")
        
    except Exception as e:
        print(f"Error al guardar el archivo: {e}")

if __name__ == "__main__":
    main()

=== Adquisición de Áreas Protegidas del Perú desde OSM ===

1. Construyendo consulta Overpass...
Consulta Overpass:

    [out:json][timeout:300][maxsize:1073741824][bbox:-18.5,-81.5,0.5,-68.5];
    (
      // Consulta específica para áreas protegidas siguiendo estándares OSM
      // Relaciones con boundary=protected_area (estándar principal)
      relation["boundary"="protected_area"];
      
      // Ways que forman límites de áreas protegidas
      way["boundary"="protected_area"];
      
      // Reservas naturales (clasificación leisure)
      relation["leisure"="nature_reserve"];
      way["leisure"="nature_reserve"];
      
      // Cualquier elemento con clasificación de protección IUCN
      relation["protect_class"~"^[1-6]$|^I[I-V]?$"];
      way["protect_class"~"^[1-6]$|^I[I-V]?$"];
      
      // Elementos específicos para Perú con designation
      relation["designation"~"Parque Nacional|Reserva Nacional|Santuario Nacional|Santuario Histórico|Reserva Paisajística|Refugio 

In [ ]:
#pip install fiona

Note: you may need to restart the kernel to use updated packages.


In [3]:
import fiona

file_path = "peru_protected_areas1.gpkg"

# Listar capas usando fiona directamente
layers = fiona.listlayers(file_path)
print("Capas disponibles:", layers)

Capas disponibles: ['peru_protected_areas1']


In [4]:
import geopandas as gpd

gdf = gpd.read_file(file_path, layer=layers[0])
print(gdf.head())

     osm_id osm_type                          name name_es designation  \
0  31127953      way  Bosque Protegido de Puengasi                       
1  31127959      way  Bosque Protegido de Puengasi                       
2  31127962      way  Bosque Protegido de Puengasi                       
3  31142752      way    Bosque Municipal Protegido                       
4  33419728      way                   Isla Santay                       

  protect_class        boundary         leisure operator website  wikidata  \
0                                nature_reserve                              
1                                nature_reserve                              
2                                nature_reserve                              
3                protected_area                                              
4                protected_area  nature_reserve                   Q7420096   

    area_km2                                           geometry  
0   0.163966  MULTIP

In [1]:
pwd

'c:\\Users\\usuario\\Documents\\GitHub\\Spatial_Analysis_of_Peru-s_Protected_Areas\\241878_hw4'